In [2]:
import altair as alt
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
train_df = pd.read_csv('MBTA_Line_and_Stop.csv')
train_df

,FID,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow
0,1,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Allston Street,place-alsgr,0,17,77,0,0,4
1,2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381
2,3,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Babcock Street,place-babck,0,151,77,0,2,8
3,4,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Back of the Hill,place-bckhl,0,36,77,0,0,4
4,5,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7916,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Porter,place-portr,59366,9940,16,3710,621,13842
7916,7917,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Quincy Adams,place-qamnl,388,26507,16,24,1657,1549
7917,7918,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Quincy Center,place-qnctr,2128,67000,16,133,4188,3182
7918,7919,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Savin Hill,place-shmnl,2292,18338,16,143,1146,7237


In [38]:
train_filt_df = train_df[train_df['total_ons'] > 1000]
train_filt_df['RugglesNEU'] = train_df['stop_name'] == 'Ruggles'
train_filt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4842 entries, 1 to 7918
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   FID                  4842 non-null   int64 
 1   mode                 4842 non-null   int64 
 2   season               4842 non-null   object
 3   route_id             4842 non-null   object
 4   route_name           4842 non-null   object
 5   direction_id         4842 non-null   int64 
 6   day_type_id          4842 non-null   object
 7   day_type_name        4842 non-null   object
 8   time_period_id       4842 non-null   object
 9   time_period_name     4842 non-null   object
 10  stop_name            4842 non-null   object
 11  stop_id              4842 non-null   object
 12  total_ons            4842 non-null   int64 
 13  total_offs           4842 non-null   int64 
 14  number_service_days  4842 non-null   int64 
 15  average_ons          4842 non-null   int64 
 16  averag

<ipython-input-38-f8b5a4f73075>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filt_df['RugglesNEU'] = train_df['stop_name'] == 'Ruggles'


In [49]:
brush = alt.selection_interval(resolve='global')

base = alt.Chart(train_filt_df).mark_line().encode(
    y=alt.Y('sum(total_ons):Q', title='Total Ons'),
    color=alt.condition(brush, 'season', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=250,
    height=250,
    title="Total number of people getting on trains in the past 3 years"
)

base_2 = alt.Chart(train_filt_df).mark_point().encode(
    y=alt.Y('total_ons:Q', title='Total Ons'),
    color=alt.condition(brush, 'season', alt.ColorValue('gray')),
    size=alt.Size('RugglesNEU', scale=alt.Scale(range = [50, 250])),
).add_selection(
    brush
).properties(
    width=250,
    height=250,
     title="Total activity on trains in the past 3 years"
)


base.encode(x=alt.X('time_period_name', title='Time Period'))| base_2.encode(x=alt.X('total_offs', title='Total Offs'))

alt.HConcatChart(...)